# Welcome!

Starting with this notebook you might either gain your first experience in Reinforcement Learning (RL) or refresh existing knowledge.

The goal is to familiarize yourself with the concept of environments (in our case `Connect 4`) and to use the relevant libraries required for this endeavor.

Throughout this notebook, you might stumble upon various errors, bugs and mistakes. Some of which might even be novel to us ;). But those should be considered as part of your learning journey, not as a hindrance.

In the end, you ideally have some running code, the respective AI player and an intuition of how RL could be used in the context of games. You might leave with more open questions than you might have entered, but this could be the prolog of your learning adventure :). To clarify, using RL for Connect 4 without further do will not produce optimal results. It will not even parallel the Minimax algorithm. Try not to use LLMs like ChatGPT unless you are really stuck at some point. We have a reference solution to the problem and provide a skeleton of what should be implemented beneath. Have fun and hopefully, we'll be working on some projects together ;).



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
import gymnasium as gym
import matplotlib.pyplot as plt
from torchinfo import summary
from rl_task import DQN, preprocess_obs
from setup import setup
from gym_env import Connect4Env

In [ ]:
setup()
env = Connect4Env(render_mode="human")
print(env.reset())

In [ ]:
#Uncomment and play a few rounds!

from play_against_random import play_against_random

%matplotlib inline

# play_against_random()

Familiarize yourself with the following functions of the environment and roughly about how the environment works.

Feel free to rerun the cell below and look at what happens

Intermediate questions: How does an environment work (at least the most important functions: step() and reset())? How might render() work with the arrays?

In [ ]:
for i in range(3):
    action = np.random.choice([0,1,2,3,4,5,6])
    print("\nAfter action", action, "\n")
    env.step(action)
    env.render()


In the following, we set up some hyperparameters. We will stumble upon those later on. Maybe you already know a few of these. If you want to know what they do or what hyperparameters in general are: Please, do not hesitate and ask ;)

In [ ]:
# --- Hyperparameters ---
GAMMA = 0.99
LR = 1e-4
BATCH_SIZE = 64
MEM_SIZE = 10000
EPS_START = 1.0
EPS_END = 0.05
EPS_DECAY = 0.99
TARGET_UPDATE = 10
EVAL_INTERVAL = 20

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

In the following, we will implement a class `ReplayBuffer`, it is supposed to store past episodes. We will randomly sample batches from those when training to prevent overfitting on recent transitions.

In [ ]:
class ReplayBuffer:

    def __init__(self, capacity):
        ''' 
        Initialize the buffer with a capacity of maxlen
        '''
        self.buffer = deque(maxlen=capacity)

    def push(self, *args):
        '''
        Push data into buffer
        '''
        self.buffer.append(args)

    def sample(self, batch_size):
        """
        randomly sample batch_size transitions from the buffer.
        return numpy arrays of each where it proves reasonable.
        """
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        # Convert to NumPy arrays (optional: convert to tensors later)
        return (
            np.array(states),
            np.array(actions),
            np.array(rewards),
            np.array(next_states),
            np.array(dones)
        )

    def __len__(self):
        '''
        return the length of the buffer
        '''
        return len(self.buffer)

Next, implement the agent class `DQNAgent`. This is our agent that interacts with the Connect 4 environment. Before continuing, please, continue with the `rl_task.py` file.

There, you will implement a first Neural Network that approximates the q-function in Connect 4. It will be the "brain" of our agent. The class `DQNAgent` will provide its "body" to perform valid actions.

In [ ]:
import torch.nn.functional as F
class DQNAgent:

    def __init__(self, obs_shape, n_actions):
        # Set the number of actions
        self.n_actions = n_actions
        self.device = DEVICE
        
        # Initalize a "Policy Net", it should be the Neural Network that is regularly updated by backpropagation
        self.policy_net = DQN(obs_shape, n_actions).to(self.device)
        
        # Initalize a target net that is a more stable Neural Network, i.e., its weights are copied from the 
        # Policy net every, e.g., 10th or 20th iteration 
        self.target_net = DQN(obs_shape, n_actions).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        # Use the Adam Optimizer, Buffer and epsilon for Epsilon Greedy Exploration
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=LR)

        # Replay buffer
        self.replay_buffer = ReplayBuffer(capacity=MEM_SIZE)

        # Epsilon-greedy parameters
        self.epsilon = EPS_START

        return

    def select_action(self, state, legal_actions, use_epsilon=True):
        # Epsilon greedy decision
        if use_epsilon and random.random() < self.epsilon:
            # Explore: Choose random legal action
            return random.choice(legal_actions)
            
        # compute the q_values
        with torch.no_grad():
            state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(DEVICE)  # shape: (1, C, H, W)
            q_values = self.policy_net(state_tensor).squeeze(0).cpu().numpy()

        # Mask illegal actions such that none of those are taken; Explicitly prohibit them for safety's sake
        # Implement masking to prevent any illegal actions from happening
        masked_q_values = np.full_like(q_values, -np.inf)
        masked_q_values[legal_actions] = q_values[legal_actions]

        # Choose the best action
        action = int(np.argmax(masked_q_values))
        
        # In case an illegal action has been taken; Exception
        if action not in legal_actions:
            raise ValueError(f"Selected illegal action: {action}")

        # Returns the selected action
        return action
    
    def optimize(self):
        """
        We calculate the target values. Those can be calculated from our target_net (the stable network). 
        To update the values, we calculate the bellman optimality equation with the target_net, to update our policy_net via backpropagation. 
        After some iterations we then also overwrite our target_net. (By backpropating the difference between target (via BOE) and the target)
        """
        # Ensure that the batch size does not exceed the memory
        if len(self.replay_buffer) < BATCH_SIZE:
            return
        # retrieve samples from memory
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(BATCH_SIZE)

        states = torch.tensor(states, dtype=torch.float32, device=self.device)
        
        # Make actions into shape [batch_size, 1] so we can use it to gather
        actions = torch.tensor(actions, dtype=torch.int64, device=self.device).unsqueeze(1)
        
        rewards = torch.tensor(rewards, dtype=torch.float32, device=self.device)
        next_states = torch.tensor(next_states, dtype=torch.float32, device=self.device)
        dones = torch.tensor(dones, dtype=torch.float32, device=self.device)

        # Get predicted Q-values for all actions in the batch of states
        q_values = self.policy_net(states)

        # Gather the Q-values corresponding to the actions that were actually taken
        state_action_values = q_values.gather(1, actions).squeeze(1)  # shape: [batch]


        with torch.no_grad():
            # Retrieve next q values and mask illegal actions for next state
            next_q_values = self.target_net(next_states)
            
            max_next_q_values = next_q_values.max(1)[0]  # max over actions

        # Bellman Optimality Equation Q(s, a) ← r + γ * max_a' Q(next_s, a')
        # set target to rewards for done states (this is needed to avoid NaN values that occur when multiplying by -inf
            
            targets = rewards + GAMMA * max_next_q_values * (1 - dones)  # no future reward if done

        # Compute loss and backpropagate it
        loss = F.mse_loss(state_action_values, targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def update_target(self):
        # Copy weights from policy_net to target_net
        self.target_net.load_state_dict(self.policy_net.state_dict())
        return

In [ ]:
# Utility function
def moving_average(data, window_size):
    """Compute moving average using a simple window."""
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')


The following method should be used to evaluate your agent's performance. Try to figure out how to fill the gaps.

In [ ]:
def evaluate_agent(agent: DQNAgent, env_name="Connect4-v0", num_games=10):
    env = gym.make(env_name, render_mode=None)
    wins = 0

    for _ in range(num_games):
        (obs, _), info = env.reset()
        done = False
        player = 0  # Agent starts

        while not done:
            legal_actions = info['legal_actions']

            if player == 0:
                # Agent's turn
                action = agent.select_action(obs, legal_actions)
            else:
                # Random opponent
                action = np.random.choice(legal_actions)

            (next_obs, _), reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            obs = next_obs
            player = 1 - player

        # Agent is player 0. If agent played last and reward == 1, it won.
        if player == 1 and reward[0] == 1:
            wins += 1

    env.close()
    return wins / num_games

Now, the hopefully final puzzle piece: Train your first Neural Network!

In [ ]:
def train_dqn(num_episodes: int = 1500):
    """
    Train the DQNAgent by having it play against a random opponent.
    """
    env = Connect4Env()              
    obs_shape = env.observation_space[0].shape
    n_actions = env.action_space.n
    agent0 = DQNAgent(obs_shape, n_actions)
    agent1 = DQNAgent(obs_shape, n_actions)

    rewards_history = []
    # Keep track of your win rates from your evaluation for the plot in the end
    winrates = []

    # baseline performance before any training
    obs, info = env.reset()
    state0 = preprocess_obs(obs[0])
    winrate = evaluate_agent(agent0)
    print(f"Evaluation after 0 episodes: winrate vs random = {winrate:.2f}")
    winrates.append(winrate)

    for episode in range(num_episodes):
        obs, info = env.reset()                           # reset returns (obs_tuple, info)
        state0 = preprocess_obs(obs[0])
        state1 = preprocess_obs(obs[1])
        done = False
        total_reward = 0.0

        while not done:
            if info['current_player'] == 0:
                # agent’s turn
                legal_actions = info['legal_actions']
                action = agent0.select_action(state0, legal_actions, use_epsilon=True)
            else:
                # opponent's turn
                legal_actions = info['legal_actions']
                action = agent1.select_action(state1, legal_actions, use_epsilon=True)

            # apply move
            (next_obs, reward, terminated, truncated, info) = env.step(action)
            done = terminated or truncated

            # if it was our move, record reward; else ignore or subtract as needed
            # if info['current_player'] == 1:
            total_reward += reward[0]

            if done:
                # get the player who lost
                if reward[0] == -1:
                    last_elem = agent0.replay_buffer.buffer[-1]
                    last_state, last_action, _, last_n_state, last_done = last_elem
                    agent0.replay_buffer.buffer[-1] = (last_state, last_action, -1, last_n_state, True)
                if reward[1] == -1:
                    last_elem = agent1.replay_buffer.buffer[-1]
                    last_state, last_action, _, last_n_state, last_done = last_elem
                    agent1.replay_buffer.buffer[-1] = (last_state, last_action, -1, last_n_state, True)
                
            next_state0 = preprocess_obs(next_obs[0])      # player-0’s view
            if info['current_player'] == 1:  # player 0 just played
                agent0.replay_buffer.push(state0, action, reward[0], next_state0, done)
                if len(agent0.replay_buffer) >= BATCH_SIZE:
                    agent0.optimize()
                    
            next_state1 = preprocess_obs(next_obs[1])      # player-1’s view
            if info['current_player'] == 0:  # player 1 just played
                agent1.replay_buffer.push(state1, action, reward[1], next_state1, done)
                if len(agent1.replay_buffer) >= BATCH_SIZE:
                    agent1.optimize()

            state0 = next_state0
            state1 = next_state1

        # after episode: decay exploration rate
        agent0.epsilon = max(EPS_END, agent0.epsilon * EPS_DECAY)
        agent1.epsilon = max(EPS_END, agent1.epsilon * EPS_DECAY)

        # sync target network periodically
        if (episode + 1) % TARGET_UPDATE == 0:
            agent0.update_target()
            agent1.update_target()

        rewards_history.append(total_reward)

        if (episode + 1) % 10 == 0:
            print(reward)
            print(f"Episode {episode + 1}, reward: {total_reward:.2f}, eps: {agent0.epsilon:.2f}")

        if (episode + 1) % EVAL_INTERVAL == 0:
            winrate = evaluate_agent(agent0)
            winrates.append(winrate)
            print(f"Evaluation after {episode + 1} episodes: winrate vs random = {winrate:.2f}")

    env.close()
    return agent0, rewards_history, winrates

Let's see its performance:

In [ ]:
setup()
#! You might want to adapt the episodes for your need.
agent, rewards, winrates = train_dqn(1000)
# Save the trained policy network
torch.save(agent.policy_net.state_dict(), "dqn_policy_net.pth")
window = 10
if len(rewards) >= window:
    rewards_ma = moving_average(rewards, window)
    plt.plot(range(window-1, len(rewards)), rewards_ma, label=f"{window}-episode moving avg")
plt.plot(rewards, alpha=0.3, label="Episode reward")
# Plot winrate
if winrates:
    plt.plot(np.arange(0, len(winrates)*EVAL_INTERVAL, EVAL_INTERVAL), winrates, label="Winrate vs random", color='red')
plt.xlabel("Episode")
plt.ylabel("Reward / Winrate")
plt.title("DQN Training Rewards and Winrate")
plt.legend()
plt.show()

#! need to restart the kernel and rerun the entire notebook when applying changes to the architecture

Now, run `play_against_rl` and see if you can defeat it

In [ ]:
from play_against_rl import play_against_agent

%matplotlib inline

play_against_agent()